# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 31 2023 9:38AM,256265,19,TR-RNG-02022023,"GCH Granules USA, Inc.",Released
1,Jan 31 2023 9:31AM,256240,15,9094379,"Brookfield Pharmaceuticals, LLC",Released
2,Jan 31 2023 9:31AM,256240,15,9094380,"Brookfield Pharmaceuticals, LLC",Released
3,Jan 31 2023 9:31AM,256240,15,9094381,"Brookfield Pharmaceuticals, LLC",Released
4,Jan 31 2023 9:31AM,256240,15,9094382,"Brookfield Pharmaceuticals, LLC",Released
5,Jan 31 2023 9:31AM,256240,15,9094383,"Brookfield Pharmaceuticals, LLC",Released
6,Jan 31 2023 9:31AM,256240,15,9094384,"Brookfield Pharmaceuticals, LLC",Released
7,Jan 31 2023 9:31AM,256240,15,9094385,"Brookfield Pharmaceuticals, LLC",Released
8,Jan 31 2023 9:31AM,256240,15,9094386,"Brookfield Pharmaceuticals, LLC",Released
9,Jan 31 2023 9:31AM,256240,15,9094387,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,256259,Released,12
18,256260,Released,3
19,256261,Released,7
20,256263,Released,2
21,256265,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256259,NaN,12.0
256260,NaN,3.0
256261,NaN,7.0
256263,NaN,2.0
256265,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256149,0.0,2.0
256172,0.0,1.0
256176,0.0,1.0
256178,1.0,0.0
256198,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256149,0,2
256172,0,1
256176,0,1
256178,1,0
256198,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256149,0,2
1,256172,0,1
2,256176,0,1
3,256178,1,0
4,256198,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256149,,2
1,256172,,1
2,256176,,1
3,256178,1,
4,256198,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 31 2023 9:38AM,256265,19,"GCH Granules USA, Inc."
1,Jan 31 2023 9:31AM,256240,15,"Brookfield Pharmaceuticals, LLC"
38,Jan 31 2023 8:49AM,256263,10,"CLINUVEL, Inc."
40,Jan 31 2023 8:41AM,256261,10,"Methapharm, Inc."
47,Jan 31 2023 8:40AM,256260,10,"Methapharm, Inc."
50,Jan 31 2023 8:38AM,256259,10,ISDIN Corporation
62,Jan 31 2023 8:34AM,256258,10,Bio-PRF
63,Jan 31 2023 8:32AM,256256,12,Hush Hush
73,Jan 31 2023 8:30AM,256230,10,"Snap Medical Industries, LLC"
84,Jan 31 2023 8:30AM,256249,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 31 2023 9:38AM,256265,19,"GCH Granules USA, Inc.",,1
1,Jan 31 2023 9:31AM,256240,15,"Brookfield Pharmaceuticals, LLC",,37
2,Jan 31 2023 8:49AM,256263,10,"CLINUVEL, Inc.",,2
3,Jan 31 2023 8:41AM,256261,10,"Methapharm, Inc.",,7
4,Jan 31 2023 8:40AM,256260,10,"Methapharm, Inc.",,3
5,Jan 31 2023 8:38AM,256259,10,ISDIN Corporation,,12
6,Jan 31 2023 8:34AM,256258,10,Bio-PRF,1,
7,Jan 31 2023 8:32AM,256256,12,Hush Hush,,10
8,Jan 31 2023 8:30AM,256230,10,"Snap Medical Industries, LLC",6,5
9,Jan 31 2023 8:30AM,256249,10,ISDIN Corporation,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 9:38AM,256265,19,"GCH Granules USA, Inc.",1,
1,Jan 31 2023 9:31AM,256240,15,"Brookfield Pharmaceuticals, LLC",37,
2,Jan 31 2023 8:49AM,256263,10,"CLINUVEL, Inc.",2,
3,Jan 31 2023 8:41AM,256261,10,"Methapharm, Inc.",7,
4,Jan 31 2023 8:40AM,256260,10,"Methapharm, Inc.",3,
5,Jan 31 2023 8:38AM,256259,10,ISDIN Corporation,12,
6,Jan 31 2023 8:34AM,256258,10,Bio-PRF,,1
7,Jan 31 2023 8:32AM,256256,12,Hush Hush,10,
8,Jan 31 2023 8:30AM,256230,10,"Snap Medical Industries, LLC",5,6
9,Jan 31 2023 8:30AM,256249,10,ISDIN Corporation,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 9:38AM,256265,19,"GCH Granules USA, Inc.",1,
1,Jan 31 2023 9:31AM,256240,15,"Brookfield Pharmaceuticals, LLC",37,
2,Jan 31 2023 8:49AM,256263,10,"CLINUVEL, Inc.",2,
3,Jan 31 2023 8:41AM,256261,10,"Methapharm, Inc.",7,
4,Jan 31 2023 8:40AM,256260,10,"Methapharm, Inc.",3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 9:38AM,256265,19,"GCH Granules USA, Inc.",1.0,NaN
1,Jan 31 2023 9:31AM,256240,15,"Brookfield Pharmaceuticals, LLC",37.0,NaN
2,Jan 31 2023 8:49AM,256263,10,"CLINUVEL, Inc.",2.0,NaN
3,Jan 31 2023 8:41AM,256261,10,"Methapharm, Inc.",7.0,NaN
4,Jan 31 2023 8:40AM,256260,10,"Methapharm, Inc.",3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 9:38AM,256265,19,"GCH Granules USA, Inc.",1.0,0.0
1,Jan 31 2023 9:31AM,256240,15,"Brookfield Pharmaceuticals, LLC",37.0,0.0
2,Jan 31 2023 8:49AM,256263,10,"CLINUVEL, Inc.",2.0,0.0
3,Jan 31 2023 8:41AM,256261,10,"Methapharm, Inc.",7.0,0.0
4,Jan 31 2023 8:40AM,256260,10,"Methapharm, Inc.",3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2818549,48.0,7.0
12,512434,10.0,1.0
15,256240,37.0,0.0
19,768753,35.0,1.0
21,512390,2.0,0.0
22,256245,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2818549,48.0,7.0
1,12,512434,10.0,1.0
2,15,256240,37.0,0.0
3,19,768753,35.0,1.0
4,21,512390,2.0,0.0
5,22,256245,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,48.0,7.0
1,12,10.0,1.0
2,15,37.0,0.0
3,19,35.0,1.0
4,21,2.0,0.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,48.0
1,12,Released,10.0
2,15,Released,37.0
3,19,Released,35.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Executing,7.0,1.0,0.0,1.0,0.0,0.0
Released,48.0,10.0,37.0,35.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Executing,7.0,1.0,0.0,1.0,0.0,0.0
1,Released,48.0,10.0,37.0,35.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Executing,7.0,1.0,0.0,1.0,0.0,0.0
1,Released,48.0,10.0,37.0,35.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()